# 1. Stimulus Generation
Generate deceptive and honest explanations using OpenAI's GPT-3.

In [ ]:
# download packages
!python -m spacy download en_core_web_lg
!pip install openai
!pip uninstall pandas
!pip install pandas==1.1.5
!pip install language-tool-python

# restart runtime

In [ ]:
# import libraries
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import random
import os
import openai

# load grammar check
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')

# load spacy
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
# load the original news headlines
df = pd.read_csv("news_items_raw.csv")

In [ ]:
df.head(2)

In [ ]:
# OpenAI key
OPENAI_API_KEY = "" # <-- insert your own API key

# Load your API key from an environment variable or secret management service
openai.api_key = OPENAI_API_KEY

In [ ]:
# The prompt for each generated explanation

# download liarplus dataset
!git clone https://github.com/Tariq60/LIAR-PLUS.git

# load liarplus dataset
liarplus_train_file = "LIAR-PLUS/dataset/jsonl/train2.jsonl"
data = []
with open(liarplus_train_file) as f:
  for line in f:
    data.append(json.loads(line))

liar_df = pd.DataFrame(data)
liar_df = liar_df.drop('id',1)
liar_df = liar_df.drop('json_file_id',1)
liar_df = liar_df.drop('topics',1)
liar_df = liar_df.drop('originator',1)
liar_df = liar_df.drop('party',1)

train = liar_df[liar_df['label'] != 'half-true']
train = train[train['label'] != 'mostly-true']
train = train[train['label'] != 'barely-true']
train = train[train['label'] != 'half-false']
train = train[train['label'] != 'pants-fire']
train = train[train['title'] == '']
train = train.drop('title',1)

# randomly select 12 items
sample_size = 12

finetune = ""
for n, i in enumerate(random.sample(list(train.index.values), sample_size)):
  veracity = 'TRUE' if train.at[i, 'label'] == 'REFUTES' else 'FALSE'
  explanation = train.at [i, 'justification'].replace('"', '').replace("'","")
  finetune += f"claim: \"{train.at[i, 'claim']}\",\nlabel: \"{veracity}\",\nexplanation: The claim is {veracity} because {explanation[0].lower() + explanation[1:]}###\n"

print(finetune)

In [ ]:
### Generate Explanations for each item based on the prompt

for i in tqdm(range(len(df))):
  for n, veracity in enumerate(["TRUE", "FALSE"]):
    prompt_ = f'{finetune}claim:\"{df["argument"].iloc[i]}\",\nlabel: \"{veracity}\",\nexplanation: The claim is {veracity} because'
    column = ["honest" if veracity == df["ground truth"].iloc[i] else "lie"][0]
    df[column+" veracity"].iloc[i] = veracity

    for j in range(3):
      ncolumn = column + str(j)

      try:
        if df["argument"].iloc[i] != "" and df.at[i+1, ncolumn] == "":
          response = openai.Completion.create(engine="davinci", prompt=prompt_, max_tokens=50, temperature=1, stop="###")["choices"][0]["text"]
          df.at[i+1, ncolumn] = response
      except Exception as e:
        print(e)
        break

In [ ]:
df.head(2)

In [ ]:
df.to_csv("stimuli_raw_news.csv")

#2. Stimulus Ranking
Stimuli were then ranked by their repeated word frequency and semantic similarity to the headline/trivia item, and correct grammar of the AI generated explanations.

In [ ]:
# load libraries
import pandas as pd
import numpy as np

In [ ]:
# load fact and logic checked dataset
df = pd.read_csv("stimuli_raw_news.csv")

In [ ]:
# Check grammar, semantic similarity, and word frequency
def cleanResponse(claim, rsp):

  # check semantic sim
  doc1 = nlp(claim)
  doc2 = nlp(rsp)
  sim = doc1.similarity(doc2)

  # count grammar errors
  matches = tool.check(rsp)
  gram = len(matches)

  # count repeated words
  words = [w for w in rsp.split(" ")]
  freq = np.mean([words.count(c) for c in claim.split(" ")])

  stats = {'sim'      : sim,
           'gram_err' : gram,
           'freq'     : freq
           }

  return stats

In [ ]:
# Run stats on stimuli
for i in tqdm(range(len(df))):
  for j in range(5):
    for veracity in ['lie','honest']:
      claim = df.at[i+1, "argument"]
      explanation = df.at[i+1, veracity+str(j)]
      stats = cleanResponse(claim, explanation)
      df.at[i+1, veracity+str(j)+'_similarity'] = stats["sim"]
      df.at[i+1, veracity+str(j)+'_gramma_err'] = stats["gram_err"]
      df.at[i+1, veracity+str(j)+'_word_freq'] = stats["freq"]

In [ ]:
# Select best stimuli
df_sorted = df.copy()

k=1
for i in range(len(df)):
  for veracity in ['lie','honest']:
    
    sample = pd.DataFrame()
    sim_best = {"val": 0,
              "vrc": "",
              "arg": "",
              "exp": ""}
              
    for j in range(3):
      sample.at[j, 'arg'] = df.at[i+1, "argument"]
      sample.at[j, 'exp'] = f"This is {df.at[i+1,veracity+' veracity']}. {df.at[i+1, veracity+str(j)]}"
      sample.at[j, 'val'] = df.at[i+1, veracity+str(j)+'_similarity'] 
      sample.at[j, 'vrc'] = veracity

      if df.at[i+1, veracity+str(j)+'_similarity'] > sim_best["val"]:
        sim_best["val"] = df.at[i+1, veracity+str(j)+'_similarity'] 
        sim_best["vrc"] = veracity
        sim_best["arg"] = df.at[i+1, "argument"]
        sim_best["exp"] = df.at[i+1, veracity+str(j)]

    sample = sample.sort_values(by=['val'], ascending=False)
    sample = sample.reset_index(drop=True)

    for j in range(len(sample)):
      df_sorted.at[k, veracity+str(j)] = sample.at[j, 'exp']

  k+=1

In [ ]:
## Clean the dataset

# remove NaN
df_sorted = df_sorted.fillna("")

# remove unfinished sentences
for veracity in ['lie', 'honest']:
  cols = [veracity+str(i) for i in range(3)]
  for col in cols:
    df_sorted[col] = df_sorted[col].apply(lambda x: x.rpartition('.')[0])

  df_sorted[veracity] = df_sorted[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
  df_sorted = df_sorted.drop(cols, 1)

# remove cols
for col in ["Unnamed: 0", "Unnamed: 9", "veracity"]:
  try:
    df_sorted = df_sorted.drop(col, 1)
  except:
    print(f"{col} not found. Skipping...")

# remove empty items
honest = df_fact_checked[df_fact_checked['honest'] != ""]
lie = df_fact_checked[df_fact_checked['lie'] != ""]
df_sorted = honest.append(lie)

In [ ]:
df_sorted.head(2)

In [ ]:
df_sorted.to_csv("stimuli_sorted_news.csv")

# 3. Fact and logic checking

Next, the stimuli are downloaded and manually checked for factual accuracy and logical validity. The criteria for **factual accuracy** were as follows: "A statement is true if and only if a credible news source within the first 2 pages of a google search states it to be so". The criteria for **logical validity** were as follows: "If the explanation can be true and the headline/trivia item false". A stimulus is rated as "unsubstantiated" if the explanation repeats the content of the headline/trivia item without providing a reason. After the fact and logic, we then exclude the logically invalid, contradicting, and unsubstantiated items.

In [ ]:
# load libraries
import pandas as pd
import numpy as np
from random import sample

In [ ]:
# load fact and logic checked dataset
df = pd.read_csv("stimuli_checked_news.csv")

In [ ]:
# remove unsubstantiated and contradictory statements
for veracity in ['lie', 'honest']:
  df = df[df[f'{veracity} logical validity'] != '']
  df = df[df[f'{veracity} logical validity'] != 'Circular']
  df = df[df[f'{veracity} veracity'] != '']
  df = df[df[f'{veracity} veracity'] != 'Self-contradiction']
  df = df[df[f'{veracity} veracity'] != 'Unsubstantiated']


# remove true lies and false honesties
df = df[df['honest veracity'] != 'FALSE']
df = df[df['lie veracity'] != 'TRUE']

In [ ]:
# Equalize the number of valid and invalid stimuli across honest and deceptive conditions

# deceptive
df_lie_valid = df[df['lie logical validity'] == 'VALID']
df_lie_invalid = df[df['lie logical validity'] == 'INVALID']

min = df[f'lie logical validity'].value_counts().min()
df_lie_valid = df_valid.sample(min)
df_lie_invalid = df_invalid.sample(min)

df = df_lie_valid.append(df_lie_invalid)

# honest
df_honest_valid = df[df['honest logical validity'] == 'VALID']
df_honest_invalid = df[df['honest logical validity'] == 'INVALID']

min = df[f'honest logical validity'].value_counts().min()
df_honest_valid = df_honest_valid.sample(min)
df_honest_invalid = df_honest_invalid.sample(min)

df = df_honest_valid.append(df_honest_invalid)

# Print results
print(f"Deceptive Condition: Distribution of Logical Validity:\n{df['lie logical validity'].value_counts()}\n")
print(f"Honest Condition: Distribution of Logical Validity:\n{df['honest logical validity'].value_counts()}\n")

In [ ]:
# Save processed dataset as final
df.to_csv("stimuli_processed_news.csv")

# Extra: Run stats on stimulus set
Check linguistic differences between lie and honest conditions

In [ ]:
# install packages
!pip install textstat

# load libraries
import numpy as np
import pandas as pd

import nltk
import textstat

from nltk.tokenize import RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# load models
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
# load processed dataset
df = pd.read_csv("stimuli_processed_news.csv")

In [ ]:
# drop irrelevant columns
df_mc = df.drop(['stimuli_ID','argument', 'ground truth', 'lie veracity','lie logical validity','honest veracity', 'honest logical validity', 'lie_flag', 'honest_flag'],1)
df_mc = pd.melt(df_mc, var_name="veracity", value_name="explanation")

In [ ]:
# Plot functions

def t_test(data, ivar, dvar):
  """Run t-test on two columns in dataset"""

  ivar_vals = [vals for vals in data[ivar].unique()]
  if len(ivar_vals) != 2:
    raise Exception('Error: Dependent variables more or less than 2!')
  
  grp_0 = data[data[ivar]==ivar_vals[0]][dvar]
  grp_1 = data[data[ivar]==ivar_vals[1]][dvar]

  t2, p2 = stats.ttest_ind(grp_0,grp_1)

  return f"t={t2}, p={p2}, ", [f"{ivar_vals[i]}_mean = {key.mean()}, {ivar_vals[i]}_std = {np.std(key)}" for i, key in enumerate([grp_0, grp_1])]



def plot_num(df, ivar, dvar, ylim):
  """Plot numeric columns in dataset"""

  fig, ax = plt.subplots(figsize=[4,8])
  
  sns.barplot(x=df[ivar], y=df[dvar], ax=ax, 
              #palette=sns.diverging_palette(15, 160, n=2)) 
              palette = ['crimson','lightskyblue'])
  ax.set_title(dvar)
  ax.set_ylim(ylim) if ylim != [None] else ""
  plt.show()

  print(t_test(df, ivar, dvar))

In [ ]:
# Compute text stats
stats = pd.DataFrame()

for key in ['lie','honest']:
  df_mc['Word Count'] = [len(tokenizer.tokenize(df_mc.at[i, "explanation"])) for i in range(len(df_mc))] # word count
  df_mc['Sentiment (TextBlob)'] = [TextBlob(df_mc.at[i, "explanation"]).sentiment.polarity for i in range(len(df_mc))] # sentiment using TextBlob
  df_mc['Sentiment (Vader)'] = [np.argmax(list(sid.polarity_scores(df_mc.at[i, "explanation"]).values())[:-1]) for i in range(len(df_mc))] # sentiment using Vader
  df_mc['Subjectivity'] = [TextBlob(df_mc.at[i, "explanation"]).sentiment.subjectivity for i in range(len(df_mc))] # opinioniated
  df_mc['Reading Ease'] = [textstat.flesch_reading_ease(df_mc.at[i, "explanation"]) for i in range(len(df_mc))] # reading ease
  df_mc['Grade Level'] = [textstat.flesch_kincaid_grade(df_mc.at[i, "explanation"]) for i in range(len(df_mc))] # grade level

# Plot text stats
columns = {'Word Count':          [None] ,
           'Sentiment (TextBlob)':[-1, 1], 
           'Sentiment (Vader)':   [0,  2],
           'Reading Ease':        [0,100],
           'Subjectivity':        [0,  1],
           'Grade Level':         [None] }

[plot_num(df_mc, "veracity", key, columns[key]) for key in columns]